<a href="https://colab.research.google.com/github/halukcansarioz/spor-analizi-calismasi/blob/main/veri_madencili%C4%9Fi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# HÜCRE 1: Gerekli Ekstra Kütüphanelerin Yüklenmesi
# Tüm modelleri (CatBoost, LightGBM vb.) çalıştırmak için gereklidir.
!pip install catboost lightgbm
print("Kurulum tamamlandı.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.3 MB/s eta 0:00:00
Kurulum tamamlandı.


In [2]:
# HÜCRE 2: Kütüphanelerin İçe Aktarılması
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Sklearn ve Model Kütüphaneleri
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor, ExtraTreesRegressor,
                              AdaBoostRegressor, GradientBoostingRegressor)
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Uyarıları ve görsel ayarları düzenle
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
print("Tüm kütüphaneler başarıyla yüklendi.")

Tüm kütüphaneler başarıyla yüklendi.


In [3]:
# HÜCRE 3: Veri Setini Yükleme ve "Gelecek Tahmini" (Forecasting) İçin Hazırlama

# 1. Dosyayı Oku
file_path = "/content/drive/MyDrive/Evaluating-the-Effectiveness-of-Machine-Learning-Models-for-Performance-Forecasting-in-Basketball/v.03_Final_state/final_data v.03/final_data v.03.csv"
print("Dosya okunuyor...")
df = pd.read_csv(file_path)

# 2. Hedef Değişkeni İsimlendir
if 'NBA_FANTASY_PTS' in df.columns:
    df = df.rename(columns={'NBA_FANTASY_PTS': 'FantasyPoints'})

# --- KRİTİK ADIM: FORECASTING (GELECEK TAHMİNİ) AYARI ---
# Veriyi Oyuncu ve Tarihe göre sıralayalım ki maçlar sıraya dizilsin
if 'GAME_DATE' in df.columns and 'PLAYER_ID' in df.columns:
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE']) # Tarih formatına çevir
    df = df.sort_values(by=['PLAYER_ID', 'GAME_DATE'])

    print("Veriler oyuncu ve tarih sırasına dizildi.")

    # Hedef Değişkeni Kaydırma (Shift)
    # Mantık: Şu anki satırdaki istatistikler (X), bir sonraki satırdaki Puanı (y) tahmin etsin.
    # Groupby kullanıyoruz ki bir oyuncunun son maçı, başka oyuncunun ilk maçına karışmasın.
    df['Target_NextGameFP'] = df.groupby('PLAYER_ID')['FantasyPoints'].shift(-1)

    print("Hedef değişken bir sonraki maça kaydırıldı (Forecasting yapısı kuruldu).")

    # Son maçı olanların "gelecek maçı" olmadığı için (NaN olur), o satırları siliyoruz
    df = df.dropna(subset=['Target_NextGameFP'])
else:
    print("HATA: Tarih veya Oyuncu ID sütunu bulunamadı, sıralama yapılamadı!")

# 3. Temizlik: Sadece Sayısal Sütunları Seç
# String (yazı) olanları atalım
numeric_df = df.select_dtypes(include=[np.number])

# NaN Değerleri Doldur
numeric_df = numeric_df.fillna(0)

# 4. X ve y Ayrımı
# X: Şu anki maçın istatistikleri (Girdi)
# y: Bir sonraki maçın puanı (Hedef)
# DİKKAT: 'FantasyPoints' sütununu X'ten çıkarmıyoruz, çünkü geçmiş performans (şu anki maç)
# gelecek maç için bir ipucudur. Sadece tahmin edilecek hedefi (Target_NextGameFP) çıkarıyoruz.
X = numeric_df.drop(['Target_NextGameFP'], axis=1)
y = numeric_df['Target_NextGameFP']

# 5. Eğitim ve Test Olarak Bölme
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nVeri Hazır (Forecasting Modu)!")
print(f"Eğitim Seti: {X_train.shape[0]} satır")
print(f"Test Seti: {X_test.shape[0]} satır")

Dosya okunuyor...
Veriler oyuncu ve tarih sırasına dizildi.
Hedef değişken bir sonraki maça kaydırıldı (Forecasting yapısı kuruldu).

Veri Hazır (Forecasting Modu)!
Eğitim Seti: 12748 satır
Test Seti: 3187 satır


In [4]:
# HÜCRE 4: WAPE (Weighted Average Percentage Error) Fonksiyonu

def calculate_wape(y_true, y_pred):
    """
    Hata Oranı Hesaplama (Daha düşük daha iyi)
    WAPE = (|Gerçek - Tahmin| Toplamı) / (|Gerçek| Toplamı) * 100
    """
    if np.sum(np.abs(y_true)) == 0:
        return 0
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true)) * 100

print("WAPE hesaplama fonksiyonu hazır.")

WAPE hesaplama fonksiyonu hazır.


In [ ]:
# HÜCRE 5: Tüm Modellerin Eğitimi (Çıktıda Açıklamalı Versiyon)

# Gerekli kütüphaneler
from sklearn.linear_model import HuberRegressor, OrthogonalMatchingPursuit
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Uyarıları filtrele
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Modellerin Tam Listesi (İsimlerin yanına parantez içinde açıklamalar eklendi)
models = {
    # --- 1. Grup: Doğrusal (Linear) ve Sağlam (Robust) Modeller ---
    "Linear Regression (Temel Doğrusal İlişki)": LinearRegression(),                                                                 # Temel model, değişkenler arası doğrusal ilişki kurar.
    "Ridge (L2 - Aşırı Öğrenme Önleyici)": Ridge(),                                                                                  # L2 regülarizasyonu ile aşırı öğrenmeyi (overfitting) engeller.
    "Lasso (L1 - Özellik Seçimi Yapar)": Lasso(),                                                                                    # L1 regülarizasyonu ile gereksiz özellikleri sıfırlar (Feature Selection).
    "Elastic Net (Ridge + Lasso Hibrit)": ElasticNet(),                                                                              # Ridge ve Lasso'nun güçlü yanlarını birleştiren hibrit model.
    "Bayesian Ridge (Olasılıksal Yaklaşım)": BayesianRidge(),                                                                        # Olasılıksal yaklaşım kullanan, belirsizliğe dayanıklı Ridge versiyonu.
    "Huber Regressor (Aykırı Değerlere Dirençli)": make_pipeline(StandardScaler(), HuberRegressor(max_iter=10000, epsilon=1.35)),    # Aykırı değerlere (outliers) karşı dirençli, hatayı minimize eden model.
    "Orthogonal Matching Pursuit (Seyrek Veri)": OrthogonalMatchingPursuit(),                                                        # Seyrek (sparse) verilerde etkili, hızlı bir yaklaşım.

    # --- 2. Grup: Mesafe ve Vektör Tabanlı Modeller ---
    "KNN (En Yakın Komşu Ortalaması)": KNeighborsRegressor(n_neighbors=9),                          # En benzer 'k' komşunun ortalamasını alarak tahmin yapar.
    "SVR (Destek Vektör - Hiper Düzlem)": SVR(kernel='rbf'),                                        # Verileri ayırmak için en uygun sınır çizgisini (hiper düzlem) bulur.

    # --- 3. Grup: Tekil Ağaç Modeli ---
    "Decision Tree (Tekil Karar Ağacı)": DecisionTreeRegressor(random_state=42),                    # Kararlar vererek dallanan tek bir ağaç yapısıdır.

    # --- 4. Grup: Topluluk (Ensemble) Modelleri ---
    "Random Forest (Bagging - Çoklu Ağaç)": RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),          # Yüzlerce ağacın ortak kararını (Bagging) kullanır.
    "Extra Trees (Daha Rastgele Orman)": ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),               # Random Forest'tan daha fazla rastgelelik içerir, varyansı düşürür.
    "AdaBoost (Sıralı Güçlendirme)": AdaBoostRegressor(random_state=42),                                                  # Zayıf modelleri sırayla güçlendirerek (Boosting) ilerler.
    "Gradient Boosting (Hata Düzeltici)": GradientBoostingRegressor(random_state=42),                                     # Hataları adım adım düzelterek güçlü bir model oluşturur.
    "XGBoost (Optimize Edilmiş Hızlı Boosting)": xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1),          # Hız ve performans için optimize edilmiş popüler Boosting kütüphanesi.
    "LightGBM (Hafif ve Hızlı Boosting)": lgb.LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbose=-1),    # Microsoft'un geliştirdiği, büyük veride çok hızlı çalışan hafif model.
    "CatBoost (Kategorik Veri Uzmanı)": CatBoostRegressor(verbose=0, random_state=42)                                     # Yandex'in geliştirdiği, özellikle kategorik verilerde çok başarılı model.
}

results = []

# --- 4. TABLO VE AÇIKLAMALAR ---

# Önce Metrik Tanımlarını Yazdıralım (Böylece tablo bozulmaz)
print("\n" + "="*100)
print("METRİK AÇIKLAMALARI (LEJANT):")
print("  * WAPE (%): Weighted Average Percentage Error (Ağırlıklı Ortalama Yüzde Hatası) -> Düşük olması iyidir.")
print("  * MAE     : Mean Absolute Error (Ortalama Mutlak Hata) -> Düşük olması iyidir.")
print("  * R2 Score: Coefficient of Determination (Açıklayıcılık Katsayısı) -> 1'e yakın olması iyidir.")
print("="*100 + "\n")

# Tablo Başlığı (Kısa ve Temiz Başlıklar)
print(f"{'Model Adı ve Açıklaması':<55} | {'WAPE (%)':<10} | {'MAE':<10} | {'R2 Score'}")
print("-" * 100)

for name, model in models.items():
    # İşlem durumunu göster (Sadece modelin kısa adını gösterelim ekran dolmasın)
    short_name = name.split('(')[0]
    print(f"--> {short_name} çalışıyor...", end="\r")

    try:
        # Eğit
        model.fit(X_train, y_train)
        # Tahmin Et
        y_pred = model.predict(X_test)

        # Metrikleri Hesapla
        wape = calculate_wape(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        results.append({"Model": name, "WAPE": wape, "MAE": mae, "R2": r2})

        # Sonucu Yazdır (Genişlik 55 karakter olarak ayarlandı)
        print(f"{name:<55} | {wape:<10.2f} | {mae:<10.2f} | {r2:.4f}   ")

    except Exception as e:
        # Hata durumunda
        print(f"{name:<55} | HATA! ({str(e)[:20]}...)   ")

print("-" * 100)
print(f"Toplam {len(results)} model başarıyla eğitildi.")


METRİK AÇIKLAMALARI (LEJANT):
  * WAPE (%): Weighted Average Percentage Error (Ağırlıklı Ortalama Yüzde Hatası) -> Düşük olması iyidir.
  * MAE     : Mean Absolute Error (Ortalama Mutlak Hata) -> Düşük olması iyidir.
  * R2 Score: Coefficient of Determination (Açıklayıcılık Katsayısı) -> 1'e yakın olması iyidir.

Model Adı ve Açıklaması                                 | WAPE (%)   | MAE        | R2 Score
----------------------------------------------------------------------------------------------------
Linear Regression (Temel Doğrusal İlişki)               | 31.88      | 11.01      | -0.0003   
Ridge (L2 - Aşırı Öğrenme Önleyici)                     | 25.00      | 8.64       | 0.3767   
Lasso (L1 - Özellik Seçimi Yapar)                       | 24.78      | 8.56       | 0.3862   
Elastic Net (Ridge + Lasso Hibrit)                      | 24.74      | 8.55       | 0.3886   
Bayesian Ridge (Olasılıksal Yaklaşım)                   | 31.50      | 10.88      | 0.0230   
Huber Regressor (Ay

In [ ]:
# HÜCRE 6: Sonuçları Görselleştirme (GÜNCEL FORECASTING SONUÇLARI)

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# 1. Sonuçları DataFrame'e Çevir ve Sırala
# (results listesinin son eğitimden geldiğinden emin olun)
results_df = pd.DataFrame(results).sort_values(by="WAPE", ascending=True)

# 2. İsimleri Kısalt (Grafik için)
results_df['ShortName'] = results_df['Model'].apply(lambda x: x.split('(')[0].strip())

# 3. En İyi Modeli Bul
best_model_row = results_df.iloc[0]
best_model_full_name = best_model_row['Model']
best_model_short_name = best_model_row['ShortName']

print(f"\n🏆 KAZANAN MODEL (Forecasting): {best_model_full_name}")
print(f"Hata Oranı (WAPE): %{best_model_row['WAPE']:.2f}")
print("="*60)

# --- GRAFİK ÇİZİMİ ---
plt.figure(figsize=(16, 8))

# SOL GRAFİK: Modellerin Hata Sıralaması
plt.subplot(1, 2, 1)
# Renk paletini 'magma_r' yaparak düşük hataları daha koyu/vurgulu yapalım
sns.barplot(x="WAPE", y="ShortName", data=results_df, palette="magma_r")

# Referans Makale Çizgisi (Kıyaslama için harika olur)
plt.axvline(x=34.14, color='red', linestyle='--', linewidth=2, label='Makale Skoru (%34.14)')
plt.legend()

plt.title("Bizim Sonuçlarımız vs. Makale (Daha Düşük Daha İyi)", fontsize=14, fontweight='bold')
plt.xlabel("WAPE (%)", fontsize=12)
plt.ylabel("")
plt.grid(axis='x', linestyle='--', alpha=0.5)

# SAĞ GRAFİK: En İyi Modelin Özellik Önemi
plt.subplot(1, 2, 2)
if best_model_full_name in models:
    best_model_instance = models[best_model_full_name]

    # Modelin içinden özellik önemlerini çekme mantığı
    final_estimator = best_model_instance
    if hasattr(best_model_instance, 'steps'): # Pipeline ise
        final_estimator = best_model_instance.steps[-1][1]

    feature_names = X.columns
    importances = None
    title_text = ""

    # Özellik Önemlerini Almaya Çalış
    if hasattr(final_estimator, 'feature_importances_'):
        importances = final_estimator.feature_importances_
        title_text = "Öznitelik Önemi (Feature Importance)"
    elif hasattr(final_estimator, 'coef_'):
        importances = np.abs(final_estimator.coef_)
        title_text = "Öznitelik Katsayıları (Coefficients)"

    if importances is not None:
        # En iyi 10 özelliği sırala
        indices = np.argsort(importances)[-10:]

        plt.title(f"Tahminde En Etkili 10 Faktör\n({best_model_short_name})", fontsize=14, fontweight='bold')
        plt.barh(range(len(indices)), importances[indices], color='teal', align='center')
        plt.yticks(range(len(indices)), [feature_names[i] for i in indices], fontsize=10)
        plt.xlabel('Etki Düzeyi', fontsize=12)
        plt.grid(axis='x', linestyle='--', alpha=0.5)
    else:
        plt.text(0.5, 0.5, "Bu model özellik önemini görselleştiremiyor.", ha='center')

plt.tight_layout()
plt.show()

In [ ]:
# HÜCRE 7: Tüm Modeller İçin Detaylı Performans Grafikleri (Sıralı Liste)

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 1. Modelleri Başarı Sırasına Göre Dizelim (En iyisi en üstte)
# (Önceki hücrelerden results_df'in geldiğini varsayıyoruz)
sorted_results = results_df.sort_values(by="WAPE", ascending=True)

# --- AYAR: KAÇ MODEL ÇİZİLSİN? ---
# Hepsini çizmek isterseniz: len(sorted_results) yapın.
# Sadece en iyi 5 tanesi için: 5 yapın.
MODEL_LIMIT = len(sorted_results)

print(f"Toplam {len(sorted_results)} model var. Sırayla {MODEL_LIMIT} tanesinin grafikleri çiziliyor...\n")

# --- GRAFİK ÇİZME FONKSİYONU ---
def plot_model_performance(model_name, model_instance, X_test, y_test, rank):

    # Tahmin Yap
    y_pred = model_instance.predict(X_test)

    # Grafik Alanı Oluştur
    fig, axes = plt.subplots(1, 3, figsize=(20, 5))
    fig.suptitle(f"#{rank} - {model_name} Performans Analizi", fontsize=16, fontweight='bold', color='darkblue')

    # --- GRAFİK 1: Gerçek vs Tahmin (Scatter) ---
    sns.scatterplot(x=y_test, y=y_pred, alpha=0.5, color='purple', edgecolor='w', ax=axes[0])
    # Referans çizgisi (y=x)
    m, b = np.polyfit(y_test, y_pred, 1)
    axes[0].plot(y_test, y_test, color='red', linestyle='--', linewidth=2, label='İdeal Tahmin (y=x)')
    axes[0].set_title("Gerçek vs Tahmin (Doğrusallık)", fontsize=12)
    axes[0].set_xlabel("Gerçek Puan")
    axes[0].set_ylabel("Tahmin Puanı")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # --- GRAFİK 2: Hata Dağılımı (Residuals) ---
    residuals = y_test - y_pred
    sns.histplot(residuals, bins=30, kde=True, color='teal', ax=axes[1])
    axes[1].axvline(x=0, color='red', linestyle='--', linewidth=2)
    axes[1].set_title("Hata Dağılımı (Sıfır Etrafında mı?)", fontsize=12)
    axes[1].set_xlabel("Hata (Gerçek - Tahmin)")

    # --- GRAFİK 3: Örneklem Takibi (Line Plot) ---
    sample_range = 50 # İlk 50 maçı göster
    axes[2].plot(range(sample_range), y_test.values[:sample_range], label='Gerçek', color='black', linewidth=2, marker='.', markersize=8)
    axes[2].plot(range(sample_range), y_pred[:sample_range], label='Tahmin', color='darkorange', linewidth=2, linestyle='--')
    axes[2].set_title(f"İlk {sample_range} Maç İçin Tahmin Uyumu", fontsize=12)
    axes[2].set_xlabel("Maç No")
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Başlık için yer ayır
    plt.show()
    print("-" * 120) # Ayırıcı çizgi

# --- DÖNGÜYÜ BAŞLAT ---
for i in range(MODEL_LIMIT):
    row = sorted_results.iloc[i]
    model_full_name = row['Model'] # Sözlükteki tam ad
    wape_score = row['WAPE']

    # Modeli sözlükten bul
    if model_full_name in models:
        model_instance = models[model_full_name]

        # Grafik fonksiyonunu çağır
        plot_model_performance(f"{model_full_name} (WAPE: %{wape_score:.2f})",
                               model_instance, X_test, y_test, i+1)

In [ ]:
# HÜCRE 8: Gelecek Puan ile Özelliklerin Korelasyonu (GARANTİLİ & HATA AYIKLAMALI)

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("--- Korelasyon Analizi Başlatılıyor ---")

# 1. Veri Kontrolü
if 'X_train' not in locals() or 'y_train' not in locals():
    print("HATA: X_train veya y_train bulunamadı! Lütfen önce HÜCRE 3'ü çalıştırın.")
else:
    # 2. Geçici DataFrame Oluştur (Güvenli Birleştirme)
    # Endeksleri sıfırlayarak birleştirelim ki kayma olmasın
    analysis_df = pd.concat([X_train.reset_index(drop=True),
                             y_train.reset_index(drop=True).rename("Hedef_GelecekPuan")],
                            axis=1)

    # Sadece sayısal verileri al (Hata önleyici)
    analysis_df = analysis_df.select_dtypes(include=[np.number])

    # 3. Korelasyon Matrisini Hesapla
    corr_matrix = analysis_df.corr()

    # Hedef değişken matriste var mı?
    if "Hedef_GelecekPuan" in corr_matrix.columns:
        # Hedef ile olan ilişkileri al (Mutlak değer olarak)
        target_corr = corr_matrix["Hedef_GelecekPuan"].drop("Hedef_GelecekPuan")

        # NaN (Boş) değerleri temizle
        target_corr = target_corr.dropna()

        if len(target_corr) == 0:
            print("UYARI: Hiçbir özellik ile korelasyon hesaplanamadı (Veriler sabit olabilir).")
        else:
            # En yüksek 10 ilişkiyi seç
            top_features = target_corr.abs().sort_values(ascending=False).head(10).index

            print(f"Grafik için seçilen en iyi {len(top_features)} özellik:")
            # Hangi özelliklerin seçildiğini ve skorlarını yazdıralım
            for feat in top_features:
                score = target_corr[feat]
                print(f" - {feat}: {score:.4f}")

            # 4. Grafiği Çiz
            plt.figure(figsize=(12, 10))
            cols_to_plot = list(top_features) + ['Hedef_GelecekPuan']

            sns.heatmap(analysis_df[cols_to_plot].corr(),
                        annot=True,
                        cmap="RdBu_r", # Kırmızı-Mavi tonları
                        fmt=".2f",
                        linewidths=.5,
                        vmin=-1, vmax=1)

            plt.title("Gelecek Maç Puanı ile En Güçlü İlişkiler", fontsize=14, fontweight='bold')
            plt.tight_layout()
            plt.show()
            print("Grafik başarıyla çizildi.")
    else:
        print("HATA: 'Hedef_GelecekPuan' sütunu korelasyon matrisinde oluşmadı.")

In [ ]:
# HÜCRE 9: Gerçek ve Tahmin Dağılımı (KDE Plot)

plt.figure(figsize=(10, 6))

# y_pred_best HÜCRE 7'den geliyor olmalı, gelmiyorsa tekrar hesapla
if 'y_pred_best' not in locals():
    y_pred_best = best_model.predict(X_test)

sns.kdeplot(y_test, label="Gerçekleşen Gelecek Puanlar", color='blue', fill=True, alpha=0.3)
sns.kdeplot(y_pred_best, label="Modelin Tahminleri", color='red', fill=True, alpha=0.3)

plt.title(f"Model: {best_model_name} - Tahmin Uyumu", fontsize=14, fontweight='bold')
plt.xlabel("Fantasy Puanı")
plt.ylabel("Yoğunluk")
plt.legend()
plt.grid(True, alpha=0.2)
plt.show()